<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/Falcon_1b_with_Biomed_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-12ygxpcl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-12ygxpcl
  Resolved https://github.com/huggingface/accelerate to commit 80da9cfb09bb3cc9f1b385cb55d6b90d025a5fd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.24.0.dev0-py3-none-any.whl size=258139 sha256=b8f92d72fb7a4fe3cea441bbcb784de9a2d6c3bb11a3a4ca9d164bf

In [2]:
from datasets import load_dataset

dataset_name = "Shushant/BiomedicalQuestionAnsweringDataset"
dataset = load_dataset(dataset_name, "train")
dataset = dataset["train"].train_test_split(test_size=3000, train_size=27000)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
import ast
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

def merge_answer(record):
  # convert string answer to dictionary
  answer = ast.literal_eval(record["answers"])["text"]
  record["answers"] = answer
  return record

# load T5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
# the context summarization will be using T5 as explored previously
summarizer = AutoModelWithLMHead.from_pretrained('t5-small', return_dict=True)

def summarize_large_context(record):
  # the max_seq_len is 512, longer than that it will be automatically cut off
  if len(torch.split(record["context"], " ")) > 512:
    input_seq = tokenizer.encode("sumarize: " + record["context"], return_tensors="pt", max_length=512, truncation=True)
    summarized_seq = model.generate(input_seq, min_length = 100, max_length = 512)
    record["context"] = tokenizer.decode(summarized_seq[0])
  return record

def generate_text_field(record):
  record['text'] = "### Context: " + record['context'] + "### Question: " + record['question'] + "### Answer: " + record['answers']
  return record

training_dataset = dataset["train"]

training_dataset.add_column(name="text", column=["temp"]*len(training_dataset))

training_dataset = training_dataset.map(merge_answer)
# training_dataset = training_dataset.map(summarize_large_context)
training_dataset = training_dataset.map(generate_text_field, remove_columns=training_dataset.column_names)


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Flattening the indices:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

In [23]:
training_dataset["text"]

['### Context: In the 6th century BCE, Taoist teacher Lao Tzu espoused a series of naturalistic concepts with some elements of humanistic philosophy. The Silver Rule of Confucianism from Analects XV.24, is an example of ethical philosophy based on human values rather than the supernatural. Humanistic thought is also contained in other Confucian classics, e.g., as recorded in Zuo Zhuan, Ji Liang says, "People is the zhu (master, lord, dominance, owner or origin) of gods. So, to sage kings, people first, gods second"; Neishi Guo says, "Gods, clever, righteous and wholehearted, comply with human." Taoist and Confucian secularism contain elements of moral thought devoid of religious authority or deism however they only partly resembled our modern concept of secularism.### Question: What is an example of Humanism based philosophy that discouraged ethics?### Answer: Silver Rule of Confucianism',
 '### Context: There were a number of avenues to upward social mobility and the achievement of no

In [4]:
import torch
import transformers
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_name = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [5]:
from peft import LoraConfig

lora_alpha = 16 # based on qlora paper
lora_dropout = 0.1 # for model up to 13B
lora_r = 64 # based on qlora paper

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)


In [6]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 16
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3 # based on qlora paper
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to=None
)


In [7]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

In [8]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [10]:
!pip install wandb

In [12]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
import wandb

wandb.init()

wandb: Currently logged in as: thangdinh75. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230919_132908-eyv9yywo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run young-bird-2
wandb: ⭐️ View project at https://wandb.ai/thangdinh75/uncategorized
wandb: 🚀 View run at https://wandb.ai/thangdinh75/uncategorized/runs/eyv9yywo


In [14]:
trainer.train()


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,8.225100
20,7.272600
30,6.774600
40,6.459100
50,6.147700
60,6.636000
70,6.355100
80,6.224200
90,6.119500
100,5.928700


KeyboardInterrupt: ignored